# Initial requirements

This notebook requires IBM Cloud Object Storage and IBM Cloud Functions
Please follow IBM Cloud dashboard and create both services.


### IBM COS Setup

Setup a bucket in IBM Cloud Object Storage
You need an IBM COS bucket which you will use to store the input data. If you don't know of any of your existing buckets or would like like to create a new one, please navigate to your cloud resource list, then find and select your storage instance. From here, you will be able to view all your buckets and can create a new bucket in the region you prefer. Make sure you copy the correct endpoint for the bucket from the Endpoint tab of this COS service dashboard. Note: The bucket names must be unique.

In [1]:
# Fill here the bucket name you created in COS Dashboard 
#bucket_name = 'kovalev-annotation-example'

In [2]:
# Define COS endpoint information. Example of US Cross Region endpoint
#cos_endpoint = 's3.private.eu-de.cloud-object-storage.appdomain.cloud'

In [3]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [4]:
import pywren_ibm_cloud
pywren_ibm_cloud.__version__

'1.0.4'

Obtain the API key and endpoint to the IBM Cloud Functions service. Navigate to Getting Started > API Key from the side menu and copy the values for "Current Namespace", "Host" and "Key" into the config below. Make sure to add "https://" to the host when adding it as the endpoint.

In [5]:
config = {
    'ibm_cf':  {'endpoint': 'https://eu-gb.functions.cloud.ibm.com',
              'namespace': 'kovalev@embl.de_dev', 
              'action_memory' : 1024,
              'api_key': '***'}, 
    'ibm_cos': {
        'endpoint': 'https://s3.eu-gb.cloud-object-storage.appdomain.cloud',
        'api_key' : '***'
    },
    'pywren' : {'storage_bucket' : 'metaspace-annotation-example'}
}

In [6]:
import json

secrets = json.load(open('config.json'))
config['ibm_cf']['api_key'] = secrets['ibm_cf']['api_key']
config['ibm_cos']['api_key'] = secrets['ibm_cos']['api_key']

### Upload metabolomics test data into COS bucket

In [7]:
import ibm_boto3
from ibm_botocore.client import Config
from ibm_botocore.client import ClientError

In [8]:
cos_client = ibm_boto3.client(service_name='s3',
                              ibm_api_key_id=config['ibm_cos']['api_key'],
#                               ibm_auth_endpoint=config['ibm_cos']['auth_endpoint'],
                              config=Config(signature_version='oauth'),
                              endpoint_url=config['ibm_cos']['endpoint'])

In [9]:
def copy(src, target_bucket, target_key):
    print('Copying from {} to {}/{}'.format(src, target_bucket, target_key))

    with open(src, "rb") as fp:
        cos_client.put_object(Bucket=target_bucket, Key=target_key, Body=fp)

    print('Copy completed for {}/{}'.format(target_bucket, target_key))

In [10]:
import os

for dirpath, dirnames, filenames in os.walk('./metabolomics'):
    for fn in filenames:
        f_path = f'{dirpath}/{fn}'
        copy(f_path, config['pywren']['storage_bucket'], f_path)

Copying from ./metabolomics/db/centroids.csv to metaspace-annotation-example/./metabolomics/db/centroids.csv
Copy completed for metaspace-annotation-example/./metabolomics/db/centroids.csv
Copying from ./metabolomics/db/formulas.csv to metaspace-annotation-example/./metabolomics/db/formulas.csv
Copy completed for metaspace-annotation-example/./metabolomics/db/formulas.csv
Copying from ./metabolomics/input/ds.txt to metaspace-annotation-example/./metabolomics/input/ds.txt
Copy completed for metaspace-annotation-example/./metabolomics/input/ds.txt
Copying from ./metabolomics/input/ds_coord.txt to metaspace-annotation-example/./metabolomics/input/ds_coord.txt
Copy completed for metaspace-annotation-example/./metabolomics/input/ds_coord.txt
Copying from ./metabolomics/data/config.json to metaspace-annotation-example/./metabolomics/data/config.json
Copy completed for metaspace-annotation-example/./metabolomics/data/config.json
Copying from ./metabolomics/data/meta.json to metaspace-annotati

In [11]:
input_data =   {'ds': 'metabolomics/input/ds.txt',
                'ds_coord': 'metabolomics/input/ds_coord.txt',
                'segments': 'metabolomics/input/segments'}
input_db =     {'centroids': 'metabolomics/db/centroids.csv',
                'formulas': 'metabolomics/db/formulas.csv'}
input_config = {'config': 'metabolomics/data/config.json',
                'meta': 'metabolomics/data/meta.json'}

# Setup

In [12]:
%config Completer.use_jedi = False
%matplotlib inline

DEBUG:matplotlib:$HOME=/home/intsco
DEBUG:matplotlib:CONFIGDIR=/home/intsco/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /opt/dev/miniconda3/envs/sm/lib/python3.6/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /opt/dev/miniconda3/envs/sm/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.0.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', 'types', 'functools', 

DEBUG:matplotlib:CACHEDIR=/home/intsco/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/intsco/.cache/matplotlib/fontlist-v300.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [13]:
from matplotlib import pyplot as plt
from scipy.sparse import coo_matrix
from collections import defaultdict
from pyImagingMSpec.image_measures import isotope_image_correlation, isotope_pattern_match
from cpyImagingMSpec import measure_of_chaos
import pywren_ibm_cloud as pywren
from itertools import chain
from pathlib import Path
import numpy as np
import pandas as pd
import pickle
import sys
import io

In [14]:
#Install PyWren-IBM
# !curl -fsSL "https://git.io/fhe9X" | sh
# try:
#     import pywren_ibm_cloud as pywren
# except:
#     !curl -fsSL "https://git.io/fhe9X" | sh
#     import pywren_ibm_cloud as pywren

### IBM Cloud Functions setup

The PyWren engine requires its server side component to be deployed in advance. This step creates a new IBM Cloud Functions function with the PyWren server side runtime. This action will be used internally by PyWren during execution phases.

In [15]:
from pywren_ibm_cloud.deployutil import clone_runtime
clone_runtime('ibmfunctions/pywren-metabolomics:3.6', config, 'pywren-ibm-cloud')

INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for namespace: kovalev@embl.de_dev
INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for host: https://eu-gb.functions.cloud.ibm.com
DEBUG:pywren_ibm_cloud.cf_connector:CF user agent set to: python-requests/2.21.0 pywren-ibm-cloud
DEBUG:pywren_ibm_cloud.cf_connector:I am about to create a new cloud function action


Cloning docker image ibmfunctions/pywren-metabolomics:3.6


DEBUG:pywren_ibm_cloud.storage.backends.cos:Set IBM COS Auth Endpoint to https://iam.cloud.ibm.com


OK --> Updated action pywren-metabolomics_3.6


INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for namespace: kovalev@embl.de_dev
INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for host: https://eu-gb.functions.cloud.ibm.com
DEBUG:pywren_ibm_cloud.cf_connector:CF user agent set to: python-requests/2.21.0 pywren-ibm-cloud
DEBUG:pywren_ibm_cloud.cf_connector:I am about to create a new cloud function action


OK --> Updated action pywren-metabolomics_3.6_modules


DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object runtimes/pywren-metabolomics_3.6.meta.json - Size: 7.0KiB - OK


Delete cloud function action: pywren-metabolomics_3.6_modules
All done!


In [16]:
pw = pywren.ibm_cf_executor(config=config, runtime='pywren-metabolomics_3.6')

INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for namespace: kovalev@embl.de_dev
INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for host: https://eu-gb.functions.cloud.ibm.com
DEBUG:pywren_ibm_cloud.cf_connector:CF user agent set to: python-requests/2.21.0 pywren-ibm-cloud
INFO:pywren_ibm_cloud.invokers:IBM Cloud Functions init for Runtime: pywren-metabolomics_3.6
DEBUG:pywren_ibm_cloud.storage.backends.cos:Set IBM COS Auth Endpoint to https://iam.cloud.ibm.com
DEBUG:pywren_ibm_cloud.runtime:Downloading runtime pre-installed modules from COS
DEBUG:pywren_ibm_cloud.runtime:Verifying Python versions
INFO:pywren_ibm_cloud.executor:IBM Cloud Functions executor created with ID b5831013-aa97


# Read Dataset Spectra

In [17]:
def parse_txt(key, data_stream, func):
    rows = []
    buffer = io.StringIO(data_stream.read().decode('utf-8'))
    while True:
        line = buffer.readline()
        if not line:
            break
        rows.append(func(line))
    return rows

In [18]:
def parse_spectrum_line(s):
    ind_s, mzs_s, int_s = s.split('|')
    return (int(ind_s),
            np.fromstring(mzs_s, sep=' ').astype('float32'),
            np.fromstring(int_s, sep=' '))

In [19]:
def parse_spectrum_coord(s):
    sp_i, x, y = map(int, s.split(','))
    return (sp_i, x, y)

In [20]:
bucket_name = config['pywren']['storage_bucket']
bucket_name

'metaspace-annotation-example'

In [21]:
iterdata = [[f'{bucket_name}/{input_data["ds"]}', parse_spectrum_line]]
# NOTE: we need to be absolutely sure that using chunk_size doesn't split a line into separate chunks
futures = pw.map(parse_txt, iterdata, chunk_size=8*1024**2)
results = pw.get_result(futures)
spectra = []
for res_list in results:
    spectra.extend(res_list)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.storage.backends.cos:Set IBM COS Auth Endpoint to https://iam.cloud.ibm.com
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
INFO:pywren_ibm_cloud.partitioner:Creating chunks from object keys...
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: None
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/103b4acb/aggdata.pickle - Size: 7.3KiB - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/103b4acb/func.pickle - Size: 1.6KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Starting function invocation: parse_txt() - Total: 11 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00000 inv

In [22]:
len(spectra)

6400

In [23]:
sp_i, mzs, ints = spectra[0]
mzs

array([ 53.904255,  55.35594 ,  55.362545,  55.949936,  56.66233 ,
        57.95044 ,  58.101448,  58.48464 ,  58.675667,  59.005196,
        59.035973,  59.161777,  59.415894,  60.146664,  60.161423,
        61.23369 ,  63.52685 ,  63.62095 ,  63.965775,  64.112434,
        65.029106,  65.40917 ,  65.61097 ,  66.66296 ,  66.77625 ,
        66.88279 ,  67.84289 ,  68.00082 ,  69.09414 ,  69.1053  ,
        70.70261 ,  71.01698 ,  71.21804 ,  71.29459 ,  71.79796 ,
        72.849205,  72.885   ,  74.43753 ,  74.84822 ,  74.984146,
        75.33112 ,  81.7692  ,  90.1316  ,  90.21486 ,  91.43951 ,
        94.44622 ,  95.16278 ,  95.54628 ,  96.42089 ,  99.29104 ,
       100.45906 , 100.77831 , 105.90295 , 111.90609 , 112.87558 ,
       115.86939 , 119.84346 , 122.13882 , 125.892654, 129.55223 ,
       135.2007  , 137.09537 , 137.28242 , 137.34137 , 138.45294 ,
       145.36913 , 145.76651 , 146.5534  , 159.07352 , 160.38416 ,
       161.22887 , 163.37315 , 163.4697  , 167.33083 , 168.375

In [24]:
ints

array([47.29862213, 50.59011841, 47.02101135, 47.47746277, 58.65240479,
       50.92343521, 42.76822662, 42.45158768, 56.05449677, 43.1148262 ,
       48.53097153, 55.81335449, 50.67507553, 43.09651566, 46.87289047,
       54.37233353, 47.54389572, 54.30453873, 47.79224777, 46.20689392,
       50.62897873, 40.02255631, 46.42947006, 43.94950485, 44.11400986,
       49.48485565, 49.11848068, 48.14740372, 40.9396019 , 41.03996277,
       44.44740295, 49.41588211, 58.40541458, 50.03430557, 43.44033432,
       48.588768  , 57.39586258, 53.97465515, 54.93380356, 84.99694061,
       49.48104477, 56.77067566, 51.75605393, 43.9488945 , 54.65294266,
       56.53967667, 60.78800964, 57.96722794, 52.62657928, 55.60599518,
       49.46104431, 58.96443176, 56.40863037, 70.75928497, 62.49911881,
       62.38255692, 52.94942093, 52.0222168 , 52.91330719, 56.34723282,
       49.91038132, 68.52725983, 56.43950272, 64.62987518, 68.28511047,
       58.55063629, 56.82152176, 65.20709992, 57.73368454, 58.89

In [25]:
iterdata = [[f'{bucket_name}/{input_data["ds_coord"]}', parse_spectrum_coord]]
futures = pw.map(parse_txt, iterdata)
spectra_coords = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.storage.backends.cos:Set IBM COS Auth Endpoint to https://iam.cloud.ibm.com
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: None
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/8e7eeae0/aggdata.pickle - Size: 560.0B - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/8e7eeae0/func.pickle - Size: 1.6KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Starting function invocation: parse_txt() - Total: 1 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00000 invoke
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Funct

In [26]:
len(spectra_coords)

6400

In [27]:
spectra_coords[:5]

[(0, 1, 1), (1, 2, 1), (2, 3, 1), (3, 4, 1), (4, 5, 1)]

# Read Molecular Database

In [28]:
def process_formulas_database(key, data_stream):
    formulas_df = pd.read_csv(data_stream._raw_stream).set_index('formula_i')
    return formulas_df.shape, formulas_df.head()

In [29]:
iterdata = [f'{bucket_name}/{input_db["formulas"]}']
futures = pw.map(process_formulas_database, iterdata)
formulas_shape, formulas_head = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.storage.backends.cos:Set IBM COS Auth Endpoint to https://iam.cloud.ibm.com
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: None
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/296d0cea/aggdata.pickle - Size: 138.0B - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/296d0cea/func.pickle - Size: 1.6KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Starting function invocation: process_formulas_database() - Total: 1 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00000 invoke
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b58

In [30]:
formulas_shape

(1149239, 1)

In [31]:
formulas_head

,formula
formula_i,
0,C100H121N50O58P9Al
1,C100H121N50O58P9As
2,C100H121N50O58P9Au
3,C100H121N50O58P9B
4,C100H121N50O58P9Ba


In [32]:
def process_centroids_database(key, data_stream):
    centroids_df = pd.read_csv(data_stream._raw_stream).set_index('formula_i')
    return centroids_df.shape, centroids_df.head(8)

In [33]:
%%time
iterdata = [f'{bucket_name}/{input_db["centroids"]}']
futures = pw.map(process_centroids_database, iterdata)
centroids_shape, centroids_head = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.storage.backends.cos:Set IBM COS Auth Endpoint to https://iam.cloud.ibm.com
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: None
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/fd36dff8/aggdata.pickle - Size: 139.0B - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/fd36dff8/func.pickle - Size: 1.6KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Starting function invocation: process_centroids_database() - Total: 1 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00000 invoke
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5

CPU times: user 124 ms, sys: 4 ms, total: 128 ms
Wall time: 9.28 s


In [34]:
centroids_shape

(4596786, 3)

In [35]:
centroids_head

,peak_i,mz,int
formula_i,,,
0,0,3255.550401,91.732506
0,1,3256.553764,100.000000
0,2,3257.557145,54.004425
0,3,3258.560492,19.359366
1,0,3303.490459,91.712252
1,1,3304.493826,100.000000
1,2,3305.497206,53.990998
1,3,3306.500550,19.354821


# Generate Ion Images

In [36]:
def real_pixel_indices(spectra_coords):
    coord_pairs = [r[1:] for r in spectra_coords]

    min_x, min_y = np.amin(np.asarray(coord_pairs), axis=0)
    max_x, max_y = np.amax(np.asarray(coord_pairs), axis=0)

    _coord = np.array(coord_pairs)
    _coord = np.around(_coord, 5)  # correct for numerical precision
    _coord -= np.amin(_coord, axis=0)

    nrows, ncols = (max_y - min_y + 1,
                    max_x - min_x + 1)

    pixel_indices = _coord[:, 1] * ncols + _coord[:, 0]
    return pixel_indices.astype(np.int32), nrows, ncols

In [37]:
def sp_df_gen(sp_it, pixel_indices):
    for sp_id, mzs, intensities in sp_it:
        for mz, ints in zip(mzs, intensities):
            yield pixel_indices[sp_id], mz, ints

In [38]:
def gen_iso_images(spectra_it, pixel_indices, centr_df, nrows, ncols, ppm, min_px=1):
    if len(centr_df) > 0:
        # a bit slower than using pure numpy arrays but much shorter
        # may leak memory because of https://github.com/pydata/pandas/issues/2659 or smth else
        sp_df = pd.DataFrame(sp_df_gen(spectra_it, pixel_indices),
                             columns=['idx', 'mz', 'ints']).sort_values(by='mz')

        # -1, + 1 are needed to extend sf_peak_mz range so that it covers 100% of spectra
        centr_df = centr_df[(centr_df.mz >= sp_df.mz.min() - 1) &
                            (centr_df.mz <= sp_df.mz.max() + 1)]
        lower = centr_df.mz.map(lambda mz: mz - mz * ppm * 1e-6)
        upper = centr_df.mz.map(lambda mz: mz + mz * ppm * 1e-6)
        lower_idx = np.searchsorted(sp_df.mz, lower, 'l')
        upper_idx = np.searchsorted(sp_df.mz, upper, 'r')

        for i, (l, u) in enumerate(zip(lower_idx, upper_idx)):
            if u - l >= min_px:
                data = sp_df.ints[l:u].values
                if data.shape[0] > 0:
                    idx = sp_df.idx[l:u].values
                    row_inds = idx / ncols
                    col_inds = idx % ncols
                    m = coo_matrix((data, (row_inds, col_inds)), shape=(nrows, ncols), copy=True)
                    yield centr_df.index[i], (centr_df.peak_i.iloc[i], m)

In [39]:
def merge_formula_images(iso_images):
    formula_images = defaultdict(list)
    for formula_i, (peak_i, img) in iso_images:
        formula_images[formula_i].append((peak_i, img))

    def filter_formula_images():
        filtered_f_images = {}
        for f_i, images in formula_images.items():
            images = sorted(images, key=lambda x: x[0])
            if len(images) > 1 and images[0][0] == 0:
                filtered_f_images[f_i] = images
        return filtered_f_images

    return filter_formula_images()

In [40]:
pixel_indices, nrows, ncols = real_pixel_indices(spectra_coords)

In [41]:
pixel_indices

array([   0,    1,    2, ..., 6397, 6398, 6399], dtype=int32)

In [42]:
nrows, ncols

(80, 80)

# Score Formula Images

In [43]:
def score_formula_images(f_images, centroids_df, empty_image):
    formula_scores = []
    for formula_i, images in f_images.items():
        centr_ints = centroids_df.loc[formula_i].int.values

        image_list = [empty_image] * len(centr_ints)
        for peak_i, img in images:
            image_list[peak_i] = img.toarray()
        flat_image_list = [img.flat[:] for img in image_list]

        m1 = isotope_pattern_match(flat_image_list, centr_ints)
        m2 = isotope_image_correlation(flat_image_list, centr_ints[1:])
        m3 = measure_of_chaos(image_list[0], nlevels=30)
        formula_scores.append([formula_i, m1, m2, m3])

    formula_scores_df = pd.DataFrame(formula_scores,
                                     columns=['formula_i', 'm1', 'm2', 'm3']).set_index('formula_i')
    formula_scores_df['msm'] = formula_scores_df.m1 * formula_scores_df.m2 * formula_scores_df.m3
    return formula_scores_df

In [44]:
empty_image = np.zeros((nrows, ncols))

In [45]:
empty_image.shape

(80, 80)

# Split Dataset into Segments

In [46]:
segm_n = 256

In [47]:
def get_segm_intervals(key, data_stream):
    centroids_df = pd.read_csv(data_stream._raw_stream).set_index('formula_i')
    segm_bounds_q = [i * 1 / segm_n for i in range(1, segm_n)]
    segm_bounds = [np.quantile(centroids_df.mz.values, q) for q in segm_bounds_q]
    segm_bounds = [0.] + segm_bounds + [sys.float_info.max]
    segm_intervals = list(zip(segm_bounds[:-1], segm_bounds[1:]))
    return segm_intervals

In [48]:
iterdata = [f'{bucket_name}/{input_db["centroids"]}']
futures = pw.map(get_segm_intervals, iterdata)
segm_intervals = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.storage.backends.cos:Set IBM COS Auth Endpoint to https://iam.cloud.ibm.com
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: None
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/a75e8782/aggdata.pickle - Size: 139.0B - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/a75e8782/func.pickle - Size: 2.0KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Starting function invocation: get_segm_intervals() - Total: 1 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00000 invoke
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-a

In [49]:
segm_intervals[:5]

[(0.0, 116.0661353472162),
 (116.0661353472162, 138.02771722325699),
 (138.02771722325699, 152.9807543117649),
 (152.9807543117649, 164.93990146661145),
 (164.93990146661145, 174.8942965607513)]

In [50]:
def iterate_over_segment(spectra, min_mz, max_mz):
    for sp_i, mzs, ints in spectra:
        smask = (mzs >= min_mz) & (mzs <= max_mz)
        yield [sp_i], mzs[smask], ints[smask]

In [51]:
def store_segm(key, data_stream, ibm_cos, segm_i, interval):
    spectra = parse_txt(key, data_stream, parse_spectrum_line)
    segm_it = iterate_over_segment(spectra, *interval)
    segm_spectra = pickle.dumps(np.array(list(segm_it)))
    ibm_cos.put_object(Bucket=bucket_name,
                       Key=f'{input_data["segments"]}/{segm_i}.pickle',
                       Body=segm_spectra)

In [52]:
%%time
iterdata = [[f"{bucket_name}/{input_data['ds']}", segm_i, segm_intervals[segm_i]]
             for segm_i in range(segm_n)]
futures = pw.map(store_segm, iterdata)
results = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.storage.backends.cos:Set IBM COS Auth Endpoint to https://iam.cloud.ibm.com
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: None
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/55fc7ba4/aggdata.pickle - Size: 73.0KiB - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/55fc7ba4/func.pickle - Size: 2.7KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Starting function invocation: store_segm() - Total: 256 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00000 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activ

DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00090 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00091 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00092 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00093 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00094 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00095 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00096 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00097 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00098 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00099 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00100 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00101 invoke
DEBUG:pywren_ibm

DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00010 - Activation ID: c0f95bceb34a43cab95bceb34a43ca39 - Time: 0.179 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00039 - Activation ID: fd1279dcca5247969279dcca527796ba - Time: 0.084 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00053 - Activation ID: ebaac6c100ed47c3aac6c100ed17c3d7 - Time: 0.043 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00016 - Activation ID: 26dcb5a63e7e42949cb5a63e7ea29416 - Time: 0.172 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00040 - Activation ID: 418a8ebbb0984c288a8ebbb0987c2890 - Time: 0.084 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00018 - Activation ID: 24cf8864fa1b4d8e8f8864fa1b7d8ee7 - Time: 0.178 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00054 - Activation ID: 3fb029e83570

DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00093 - Activation ID: b63b10320c5c4de4bb10320c5c8de457 - Time: 0.044 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00091 - Activation ID: 83f67124949046d7b671249490a6d7f6 - Time: 0.051 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00096 - Activation ID: 8eae780d2cec4b1dae780d2ceccb1d1b - Time: 0.042 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00090 - Activation ID: 847e2a83d1074b69be2a83d107db69c1 - Time: 0.058 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00098 - Activation ID: a92d34cfb2d041e5ad34cfb2d0d1e585 - Time: 0.040 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00092 - Activation ID: 9aa7e93edd064889a7e93edd064889f4 - Time: 0.053 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00097 - Activation ID: 8c553b7d7470

DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00060 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00061 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00062 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00031 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00064 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00066 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00050 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00034 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00044 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00068 complete
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00127 - Activation ID: 69e952eddefe4de0a952eddefeede010 - Time: 0.046 seconds
DEBUG:pywre

...

DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00139 - Activation ID: 1d7383fde5e64cc7b383fde5e6fcc79e
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00140 - Activation ID: 19f74736eaa64902b74736eaa6990202
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00141 - Activation ID: e514ba7cfcc243f494ba7cfcc233f489
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00142 - Activation ID: 9f095cb413104ebf895cb413101ebf50
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00143 - Activation ID: 9538d2e6838c46acb8d2e6838cb6acac
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00144 - Activation ID: 905af15a68b842309af15a68b862302c
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00145 - Activation ID: 0058fc0a956740ff98fc0a956710ff3c
DEBUG:pywren_ibm_cloud.future:Executor ID

DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00199 - Activation ID: 43b73d1884dc4ef7b73d1884dc9ef7c4 - Time: 6.74 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00200 - Activation ID: a326f67754ce4bc8a6f67754ce6bc852 - Time: 77.90 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00201 - Activation ID: 495f61e92438427d9f61e92438627d62 - Time: 6.58 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00202 - Activation ID: 17830ba15e414701830ba15e41b70119 - Time: 6.60 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00204 - Activation ID: e0f002767f7c4073b002767f7cf073f4 - Time: 8.89 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00203 - Activation ID: 09abb02ddc3f4c6babb02ddc3ffc6b18 - Time: 6.75 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Respons

DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00196 - Activation ID: 2ded9449ca73486fad9449ca73386f43
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00252 - Activation ID: eee0ee86aaf44714a0ee86aaf4371485 - Time: 5.99 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00253 - Activation ID: 7ad152f3368e49229152f3368e5922c6 - Time: 6.47 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00254 - Activation ID: aedbef8c4fd345d59bef8c4fd3c5d579 - Time: 7.10 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00255 - Activation ID: f86723b5b05c43e3a723b5b05c83e3d9 - Time: 6.11 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00197 - Activation ID: e86ae22b77b74d17aae22b77b7cd173e
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00198 - Activation ID

DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00252 - Activation ID: eee0ee86aaf44714a0ee86aaf4371485
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00253 - Activation ID: 7ad152f3368e49229152f3368e5922c6
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00254 - Activation ID: aedbef8c4fd345d59bef8c4fd3c5d579
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00255 - Activation ID: f86723b5b05c43e3a723b5b05c83e3d9
INFO:pywren_ibm_cloud.wren:Executor ID b5831013-aa97 Finished getting results


CPU times: user 4.34 s, sys: 320 ms, total: 4.66 s
Wall time: 2min 15s


# Annotation Pipeline Applied to each Segment in Parallel

In [53]:
def filter_formula_images(formula_images, formula_scores_df):
    return {f_i: images
            for (f_i, images) in formula_images.items()
            if f_i in formula_scores_df.index}

In [54]:
def annotate_segm_spectra(key, data_stream, ibm_cos):
    print("Welcome to annotate spectra")
    centroids_stream = ibm_cos.get_object(Bucket=bucket_name, Key=input_db['centroids'])['Body']
    print("Read centroids DB from IBM COS")
    centroids_df = pd.read_csv(centroids_stream).set_index('formula_i')
    print("Create Pandas DF for centroids DB")
    spectra = pickle.loads(data_stream.read())
    iso_images = gen_iso_images(spectra, pixel_indices, centroids_df, nrows, ncols, ppm=3)
    formula_images = merge_formula_images(list(iso_images))
    formula_scores_df = score_formula_images(formula_images, centroids_df, empty_image)
    formula_scores_df = formula_scores_df[formula_scores_df.msm > 0]
    formula_images = filter_formula_images(formula_images, formula_scores_df)
    return formula_scores_df, formula_images 

In [55]:
%%time
iterdata = [f'{bucket_name}/{input_data["segments"]}/{segm_i}.pickle' for segm_i in range(segm_n)]
futures = pw.map(annotate_segm_spectra, iterdata)
results = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.storage.backends.cos:Set IBM COS Auth Endpoint to https://iam.cloud.ibm.com
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: {'/opt/dev/miniconda3/envs/sm/lib/python3.6/site-packages/pyImagingMSpec'}
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/f151c79b/aggdata.pickle - Size: 36.9KiB - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/b5831013-aa97/f151c79b/func.pickle - Size: 174.6KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Starting function invocation: annotate_segm_spectra() - Total: 256 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Acti

DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00089 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00090 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00091 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00092 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00093 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00094 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00095 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00096 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00097 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00098 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00099 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00100 invoke
DEBUG:pywren_ibm

DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00044 - Activation ID: b1c5ac9c25aa481585ac9c25aa38152b - Time: 0.037 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00028 - Activation ID: 98331709753d4e57b31709753dae577a - Time: 0.090 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00042 - Activation ID: 6a608c6f5da0411aa08c6f5da0111a75 - Time: 0.046 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00048 - Activation ID: 559b0ad23d7d48eb9b0ad23d7d68ebc6 - Time: 0.038 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00025 - Activation ID: af3278d43df6423fb278d43df6d23f0f - Time: 0.112 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00049 - Activation ID: 0f0ee0ab94d84e928ee0ab94d88e9208 - Time: 0.040 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00050 - Activation ID: ffdcecf28c5f

DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00092 - Activation ID: 78096a446ebc43e7896a446ebca3e7ef - Time: 0.042 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00094 - Activation ID: 468fbc1801434a718fbc1801437a71ac - Time: 0.038 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00090 - Activation ID: 4fe7cf2073e34feaa7cf2073e36fea0f - Time: 0.049 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00095 - Activation ID: 0e9c27008f8444319c27008f845431de - Time: 0.039 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00096 - Activation ID: dc0f01377e074c468f01377e077c46bf - Time: 0.037 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00083 - Activation ID: 4fd76f1585834663976f158583d66377 - Time: 0.083 seconds
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00098 - Activation ID: caf25bd60caf

DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00049 complete
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID b5831013-aa97 Function 00125 - Activation ID: 2a1bc1145b7b48ee9bc1145b7b48eeef - Time: 0.040 seconds
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00050 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00169 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00128 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00018 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00014 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00053 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00055 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00031 complete
DEBUG:pywren_ibm_cloud.executor:Executor ID b5831013-aa97 Activation 00057 complete
DEBUG:pywren_ib

...

DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00068 - Activation ID: b5e62e92da9644daa62e92da96c4da00
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00096 - Activation ID: dc0f01377e074c468f01377e077c46bf
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00090 - Activation ID: 4fe7cf2073e34feaa7cf2073e36fea0f
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00099 - Activation ID: 70f43be687a849c7b43be687a8d9c72d
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00080 - Activation ID: 3ca762b88ab94a1aa762b88ab9ea1ac9
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00120 - Activation ID: c37c984aa1194644bc984aa119f644f1
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00047 - Activation ID: 0c843204380d49cc843204380d29cce7
DEBUG:pywren_ibm_cloud.future:Executor ID

DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00037 - Activation ID: 0065dd66210a4ca9a5dd66210abca9e1
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00059 - Activation ID: c8f674ef6a3f4ad0b674ef6a3f6ad071
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00117 - Activation ID: b8411b40739a482a811b40739af82a28
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00111 - Activation ID: 641147f142ea44b59147f142eae4b53c
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00079 - Activation ID: c7fb6ae56c3d485dbb6ae56c3d985d98
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00100 - Activation ID: 578af373b2e04fc88af373b2e04fc88b
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00118 - Activation ID: 124f9b6a31ec46ed8f9b6a31ec26ed1b
DEBUG:pywren_ibm_cloud.future:Executor ID

DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00041 - Activation ID: 07972a503d3843e5972a503d38b3e5dd - Time: 35.09 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00031 - Activation ID: 1cab6a3dd2e449fdab6a3dd2e419fd3c - Time: 34.63 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00045 - Activation ID: 38e3b0dfda0847fda3b0dfda0887fda1 - Time: 35.87 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00043 - Activation ID: 8d1ee7dd75bd4b539ee7dd75bd4b530b - Time: 34.37 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00049 - Activation ID: 0f0ee0ab94d84e928ee0ab94d88e9208 - Time: 34.20 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00032 - Activation ID: 0b1c80bd79b14df29c80bd79b1ddf2c9 - Time: 36.64 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Go

DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00025 - Activation ID: af3278d43df6423fb278d43df6d23f0f
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00014 - Activation ID: a127214b761b4bf7a7214b761bdbf78d
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00006 - Activation ID: fa182070d4314cb4982070d431acb475
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00010 - Activation ID: 928c62701f844d768c62701f842d769f
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00016 - Activation ID: 94a0cf83733f496fa0cf83733f496f59
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Got output from Function 00019 - Activation ID: 5847e23d4351464087e23d4351a640cc
DEBUG:pywren_ibm_cloud.future:Executor ID b5831013-aa97 Response from Function 00004 - Activation ID: a2158ecc94ea44d3958ecc94eaa4d33c - Time: 47.66 seconds
DEBUG:pywren_ibm_clou

ReadTimeoutError: Read timeout on endpoint URL: "None"

In [56]:
len(results)

256

# Get Results

In [57]:
formula_scores_list, formula_images_list = list(zip(*results))
formula_scores_df = pd.concat(formula_scores_list)
formula_images = dict(chain(*[segm_formula_images.items()
                              for segm_formula_images in formula_images_list]))

TypeError: zip argument #1 must support iteration

In [ ]:
img = formula_images[896952][0][1]
plt.imshow(img.toarray())

## Clean Segments Datasets

In [ ]:
def clean_segments_datasets(bucket, key, data_stream, ibm_cos):
    ibm_cos.delete_object(Bucket=bucket, Key=key)

In [ ]:
%%time
data_stream = f'{bucket_name}/{input_data["segments"]}'
futures = pw.map(clean_segments_datasets, data_stream)
pw.get_result(futures)

In [ ]:
pw.clean()